In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers

In [ ]:
from transformers import XLMRobertaTokenizer, TFXLMRobertaModel


In [ ]:
!pip install sentencepiece

In [ ]:
model_name = "xlm-roberta-base"

tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)

In [ ]:


bert_model = TFXLMRobertaModel.from_pretrained(model_name)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFXLMRobertaModel: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias']
- This IS expected if you are initializing TFXLMRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFXLMRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFXLMRobertaModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaModel for predictions without further training.


In [ ]:
# Model Saving
save_model = False
model_weights_location = '/content/drive/MyDrive/hate rate/task 4/model saving/xlm-2lstm-2dense only/model_weights.pth'
model_architecture_location = "/content/drive/MyDrive/hate rate/task 4/model saving/xlm-2lstm-2dense only/model_architecture.pth"

# Model Training
num_epochs = 2


# Data Location
data_file_location = '/content/drive/MyDrive/hate rate/task 4/dataset/train_AH_preprocessed.csv'

# Fine-Tuning
fine_tune_bert = False

# Tokenization
max_token_length = 190

# LSTM Configuration
first_lstm_bidirectional = True

# Batch Size
batch_size = 8

# Trained Model Usage
use_saved_model = False
saved_model_weight_location = ''

# Binary Classification Threshold
classification_threshold = 0.5

# Save Final Model
save_final_model = False
final_model_weight_location = ''
final_model_architecture_location = ''

# Save Training/Validation Accuracy Dataframe
save_accuracy_dataframe = False
accuracy_dataframe_location = ''

# Test Data Usage
validation_data_percentage = 0.0
use_test_data = False

# Current BERT Model
model_name = "xlm-roberta-base"


max_tokenization_length=128

In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv(data_file_location)

In [ ]:
#convert label to int
output=pd.Series()
for indx,row in df.iterrows():
  current=row['task_1']
  if(current=="NOT"):
    output.at[indx]=0
  else:
    output.at[indx]=1
df['output']=output


#finding longest sentence
maxx=0

for s in df['text']:
  s_len=len(s.split())
  if(s_len>maxx):
    sent=s
  maxx=max(s_len,maxx)


print(maxx)
# Pad sequences to a fixed length (adjust MAX_LEN if needed)
MAX_LEN = maxx

<ipython-input-11-5acdf9bf36d9>:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  output=pd.Series()


66


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split



# Splitting the dataframe into train and test sets
# _, datadf = train_test_split(df, test_size=0.2, random_state=42)

train_df=df
#train_df,data_df=train_test_split(df,test_size=validation_data_percentage,random_state=50)

# if(use_test_data):
#   val_df,test_df=train_test_split(data_df,test_size=0.5,random_state=100)
#   test_df=test_df.reset_index(drop=True)
# else:
#   val_df=data_df




#train_df,test_df=train_test_split(data_df,test_size=0.2,random_state=50)

# Reset the index of the DataFrame
train_df = train_df.reset_index(drop=True)
# Reset the index of the DataFrame
##val_df = val_df.reset_index(drop=True)



In [ ]:
for layer in bert_model.layers:
    layer.trainable = False



In [ ]:
df.columns

Index(['Unnamed: 0', 'S. No.', 'text', 'task_1', 'preprocessed_text',
       'output'],
      dtype='object')

In [ ]:

def get_embeddings(df):
  train_input_ids = []
  train_attention_masks = []
  train_labels = []
  for idx, row in df.iterrows():
    #text=row['preprocessed_text']
    text=row['text']
    if(text=='None'):
      text=''
    if(text=='nan'):
      text=''
    label=row['output']

    text_token=tokenizer(text,truncation=True,padding='max_length',max_length=180)
    train_input_ids.append(text_token['input_ids'])
    train_attention_masks.append(text_token['attention_mask'])
    train_labels.append(label)

  return train_input_ids,train_attention_masks,train_labels

    # text_input=[text_token['input_ids'],text_token['attention_mask']]


In [ ]:
train_input_ids,train_attention_masks,train_labels=get_embeddings(train_df)



In [ ]:
val_input_ids,val_attention_masks,val_labels=get_embeddings(val_df)

In [ ]:
# Convert to numpy arrays
import numpy as np
train_input_ids = np.array(train_input_ids)
train_attention_masks = np.array(train_attention_masks)
train_labels = np.array(train_labels)



In [ ]:

val_input_ids = np.array(val_input_ids)
val_attention_masks = np.array(val_attention_masks)
val_labels = np.array(val_labels)

In [ ]:
train_input_ids.shape
val_input_ids.shape


(404, 128)

In [ ]:
import tensorflow as tf

In [ ]:
#this one is working fine
train_dataset = tf.data.Dataset.from_tensor_slices(
    ({'input_ids': train_input_ids, 'attention_mask': train_attention_masks}, train_labels)
).batch(batch_size)


In [ ]:
val_dataset = tf.data.Dataset.from_tensor_slices(
    ({'input_ids': val_input_ids, 'attention_mask': val_attention_masks}, val_labels)
).batch(batch_size)


In [ ]:
import tensorflow as tf
from transformers import TFXLMRobertaModel

# Define model configuration
max_seq_length = 180


lstm1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=128, return_sequences=True, return_state=True))
lstm2 = tf.keras.layers.LSTM(units=64, return_sequences=True, return_state=True)
#lstm3 = tf.keras.layers.LSTM(units=64, return_sequences=True, return_state=True)

dense_layers = tf.keras.Sequential([
    tf.keras.layers.Dense(units=32, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(units=1)
])

sigmoid = tf.keras.layers.Activation('sigmoid')

# Define input layers
input_dict={
'input_ids':tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='input_ids'),
'attention_mask':tf.keras.layers.Input(shape=(max_seq_length,), dtype=tf.int32, name='attention_mask')}

# # XLM-Roberta outputs
# xlm_output = xlm_roberta([input_ids, attention_mask])[0]
# XLM-Roberta outputs
xlm_output = bert_model([input_dict['input_ids'], input_dict['attention_mask']])[0]

# LSTM 1
lstm_output, forward_hidden_state, forward_cell_state, backward_hidden_state, backward_cell_state= lstm1(xlm_output)

# LSTM 2 (feeding output of LSTM 1)
_,lstm2_output, _ = lstm2(lstm_output)

# Dense layers
dense_output = dense_layers(lstm2_output)

output = sigmoid(dense_output)
output = tf.squeeze(output, axis=1)

#model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
# Define the model
model = tf.keras.Model(inputs=input_dict, outputs=output)
# Move the model to GPU if available
# if tf.config.list_physical_devices('GPU'):
#     model = tf.keras.utils.multi_gpu_model(model, gpus=2)

# Compile the model


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

n_epochs=100
#optimizer = tf.keras.optimizers.legacy.AdamW(learning_rate=0.1, decay=0.001)
optimizer=tf.keras.optimizers.AdamW(
    learning_rate=0.0001,
    weight_decay=0.004)

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Create ReduceLROnPlateau callback
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', mode='min', factor=0.7, patience=1, verbose=1)

# Train the model using fit method
history = model.fit(
    train_dataset,  # Replace with your training dataset
    epochs=n_epochs,
    callbacks=[lr_scheduler],
    batch_size=32,
    verbose=1,

    use_multiprocessing=True

)


Epoch 1/100
505/505 [==============================] - ETA: 0s - loss: 0.6814 - accuracy: 0.1347

505/505 [==============================] - 97s 148ms/step - loss: 0.6814 - accuracy: 0.1347 - lr: 1.0000e-04
Epoch 2/100
505/505 [==============================] - ETA: 0s - loss: 0.6797 - accuracy: 0.1149

505/505 [==============================] - 74s 146ms/step - loss: 0.6797 - accuracy: 0.1149 - lr: 1.0000e-04
Epoch 3/100
505/505 [==============================] - ETA: 0s - loss: 0.6810 - accuracy: 0.1644

505/505 [==============================] - 74s 146ms/step - loss: 0.6810 - accuracy: 0.1644 - lr: 1.0000e-04
Epoch 4/100
505/505 [==============================] - ETA: 0s - loss: 0.6729 - accuracy: 0.1545

505/505 [==============================] - 74s 146ms/step - loss: 0.6729 - accuracy: 0.1545 - lr: 1.0000e-04
Epoch 5/100
505/505 [==============================] - ETA: 0s - loss: 0.6649 - accuracy: 0.1663

...

505/505 [==============================] - 74s 146ms/step - loss: 0.6391 - accuracy: 0.1960 - lr: 1.0000e-04
Epoch 12/100
505/505 [==============================] - ETA: 0s - loss: 0.6377 - accuracy: 0.1762

505/505 [==============================] - 74s 146ms/step - loss: 0.6377 - accuracy: 0.1762 - lr: 1.0000e-04
Epoch 13/100
505/505 [==============================] - ETA: 0s - loss: 0.6328 - accuracy: 0.1525

505/505 [==============================] - 74s 146ms/step - loss: 0.6328 - accuracy: 0.1525 - lr: 1.0000e-04
Epoch 14/100
505/505 [==============================] - ETA: 0s - loss: 0.6307 - accuracy: 0.1525

505/505 [==============================] - 74s 146ms/step - loss: 0.6307 - accuracy: 0.1525 - lr: 1.0000e-04
Epoch 15/100
505/505 [==============================] - ETA: 0s - loss: 0.6290 - accuracy: 0.1782

505/505 [==============================] - 74s 146ms/step - loss: 0.6290 - accuracy: 0.1782 - lr: 1.0000e-04
Epoch 16/100
503/505 [============================>.] - ETA: 0s - loss: 0.6335 - accuracy: 0.1610